# Getting started with machine learning pipelines

## Machine Learning Pipelines

At the core of the pyspark.ml module are the Transformer and Estimator classes. Almost every other class in the module behaves similarly to these two basic classes.

Transformer classes have a .transform() method that takes a DataFrame and returns a new DataFrame; usually the original one with a new column appended. For example, you might use the class Bucketizer to create discrete bins from a continuous feature or the class PCA to reduce the dimensionality of your dataset using principal component analysis.

Estimator classes all implement a .fit() method. These methods also take a DataFrame, but instead of returning another DataFrame they return a model object. This can be something like a StringIndexerModel for including categorical data saved as strings in your models, or a RandomForestModel that uses the random forest algorithm for classification or regression.

### Join the DataFrames

In [ ]:
planes = planes.withColumnRenamed("year", "plane_year")
model_data = flights.join(planes, on="tailnum", how = "leftouter")

## Data types


Use the .cast() method in combination with the .withColumn() method to change data types of columns. It's important to note that .cast() works on columns, while .withColumn() works on DataFrames.

The only argument you need to pass to .cast() is the kind of value you want to create, in string form. For example, to create integers, you'll pass the argument "integer" and for decimal numbers you'll use "double".

### String to integer

In [ ]:
model_data = model_data.withColumn("arr_delay", model_data.arr_delay.cast("integer"))
model_data = model_data.withColumn("air_time", model_data.air_time.cast("integer"))
model_data = model_data.withColumn("month", model_data.month.cast("integer"))
model_data = model_data.withColumn("plane_year", model_data.plane_year.cast("integer"))

### Create a new column

In [ ]:
model_data = model_data.withColumn("plane_age", model_data.year - model_data.plane_year)

### Making a Boolean

In [ ]:
model_data = model_data.withColumn("is_late", model_data.arr_delay > 0)
model_data = model_data.withColumn("label", model_data.is_late.cast("integer"))

model_data = model_data.filter("arr_delay is not NULL and dep_delay is not NULL and air_time is not NULL and\
                                plane_year is not NULL")

## Strings and factors

With pyspark.ml.features submodule you can create what are called 'one-hot vectors' which is a way of representing a categorical feature where every observation has a vector in which all elements are zero except for at most one element, which has a value of one (1).

The first step to encoding your categorical feature is to create a StringIndexer. Members of this class are Estimators that take a DataFrame with a column of strings and map each unique string to a number. Then, the Estimator returns a Transformer that takes a DataFrame, attaches the mapping to it as metadata, and returns a new DataFrame with a numeric column corresponding to the string column.

The second step is to encode this numeric column as a one-hot vector using a OneHotEncoder. This works exactly the same way as the StringIndexer by creating an Estimator and then a Transformer. The end result is a column that encodes your categorical feature as a vector.

### Carrier

The inputCol is the name of the column you want to index or encode, and the outputCol is the name of the new column that the Transformer should create.



In [ ]:
carr_indexer = StringIndexer(inputCol = "carrier", outputCol="carrier_index")
carr_encoder = OneHotEncoder(inputCol = "carrier_index", outputCol="carrier_fact")

### Destination

In [ ]:
dest_indexer = StringIndexer(inputCol = "dest", outputCol="dest_index")
dest_encoder = OneHotEncoder(inputCol = "dest_index", outputCol="dest_fact")

### Assemble a vector

The last step in the Pipeline is to combine all of the columns containing our features into a single column. This has to be done before modeling can take place because every Spark modeling routine expects the data to be in this form. You can do this by storing each of the values from a column as an entry in a vector. Then, from the model's point of view, every observation is a vector that contains all of the information about it and a label that tells the modeler what value that observation corresponds to.

VectorAssembler transformer takes all of the columns you specify and combines them into a new vector column.

In [ ]:
vec_assembler = VectorAssembler(inputCols=["month", "air_time", "carrier_fact", "dest_fact", "plane_age"], outputCol ="features")

### Create the pipeline

Pipeline is a class in the pyspark.ml module that combines all the Estimators and Transformers that you've already created. This lets you reuse the same modeling process over and over again by wrapping it up in one simple object. 

In [ ]:
from pyspark.ml import Pipeline

flights_pipe = Pipeline(stages = [dest_indexer, dest_encoder, carr_indexer, carr_encoder, vec_assembler])

## Test vs Train


One of the most important steps is to split the data into a test set and a train set.

### Transform the data

In [ ]:
piped_data = flights_pipe.fit(model_data).transform(model_data)

### Split the data

In [ ]:
training, test = piped_data.random_Split([0.6, 0.4])